In [3]:
import os
import matplotlib
matplotlib.use("Agg")

from keras.preprocessing.image import ImageDataGenerator
from keras.optimizers import SGD
from sklearn.preprocessing import LabelBinarizer
from sklearn.metrics import classification_report
import matplotlib.pyplot as plt
import numpy as np

Using TensorFlow backend.


In [4]:
# DATA PATHS

TRAIN_FOLDER_PATH = '../../data/train/'
VAL_FODLER_PATH = '../../data/validation/'
TEST_FOLDER_PATH = '../../data/test/'
PREVIEW_IMAGES_FOLDER = '../../data/preview/'

TRAIN_CSV_PATH = '../../data/train_labels.csv'
VALIDATION_CSV_PATH = '../../data/validation_labels.csv'

In [5]:
# Model name used to save the weights in h5 format. The name match with the Jupyter notebook

MODEL_NAME = 'model_1'

In [6]:
def csv_image_generator(inputPath, bs, lb, mode='train', aug=None):
    # open the CSV file for reading
    f = open(inputPath, "r")
    
    while True:
        # initialize the batches of images and labels
        images = []
        labels = []
        
        # keep looping until we reach the batch size
        while len(images) < bs:
            line = f.readline()
            
            # check if EOF
            if line == "":
                # reset the file pointer to the beginning of the file and re-read the line
                f.seek(0)
                line = f.readline()
                
                # if we are evaluating we should now break from our
                # loop to ensure we don't continue to fill up the
                # batch from samples at the beginning of the file
                if mode == "eval":
                    break
                    
            # extract the label and construct the image
            line = line.strip().split(",")
            label = line[1]
            image = np.array([int(x) for x in line[0:]], dtype="uint8")
            image = image.reshape((64,64,3))
            
            # update the batches lists
            images.append(image)
            labels.append(label)
            
            # one-hot encode the labels
            labels = lb.transform(np.array(labels))
            
            if aug is not None:
                (images, labels) = next(aug.flow(np.array(images), labels, batch_size=bs))
            
            yield(np.array(images), labels)

In [7]:
NUM_EPOCHS = 75
BATCH_SIZE = 32

NUM_TRAIN_IMAGES = 2756
NUM_VALIDATION_IMAGES = 552

In [8]:
# open the training CSV file, then initialize the unique set of class
# labels in the dataset along with the testing labels
f = open(TRAIN_CSV_PATH, "r")
labels = set()
validationLabels = []

for line in f:
    label = line.strip().split(",")[1]
    labels.add(label)
    NUM_TRAIN_IMAGES += 1

f.close()

f = open(VALIDATION_CSV_PATH, "r")

for line in f:
    label = line.strip().split(",")[1]
    validationLabels.append(label)
    NUM_VALIDATION_IMAGES += 1

f.close()

In [9]:
# create the label binarizer for one-hot encoding labels, then encode
# the testing labels
lb = LabelBinarizer()
lb.fit(list(labels))
validationLabels = lb.transform(validationLabels)

aug = ImageDataGenerator(rotation_range = 20, horizontal_flip=True, fill_mode="nearest")

In [10]:
# initialize both the training and validation image generators
trainGen = csv_image_generator(TRAIN_CSV_PATH, BATCH_SIZE, lb, mode="train", aug=aug)
validationGen = csv_image_generator(VALIDATION_CSV_PATH, BATCH_SIZE, lb, mode="train", aug=None)

In [13]:
# Initialize the  model and compile it
import architecture_1

arc1 = architecture_1

model = arc1.get_model()
opt = SGD(lr=1e-2, momentum=0.9, decay=1e-2 / NUM_EPOCHS)
model.compile(loss="binary_crossentropy", optimizer=opt, metrics=["accuracy"])

# train the network
print("[INFO] training w/ generator...")
H = model.fit_generator(trainGen,
                       steps_per_epoch=NUM_TRAIN_IMAGES // BATCH_SIZE,
                       validation_data=validationGen,
                       validation_steps=NUM_VALIDATION_IMAGES // BATCH_SIZE,
                       epochs=NUM_EPOCHS)

[INFO] training w/ generator...
Instructions for updating:
Use tf.cast instead.
Epoch 1/75


ValueError: invalid literal for int() with base 10: 'image_id'